Implémentation basique de l'algorithme `greedy best first search` en python sur le graphe suivant pour aller de `A` à `H`.

```mermaid
graph LR
    A -- 1 --> B
    A -- 2 --> C
    A -- 3 --> E

    B -- 1 --> D
    B -- 4 --> E

    E -- 2 --> H
    E -- 1 --> D

    C -- 4 --> F
    C -- 2 --> G
    
    D -- 5 --> H
``` 

In [118]:
class Node:
    """
    Définit un noeud dans un graphe avec un nom et un coût.

    Attributes
    ----------
        name (str): Nom du noeud.
        cost (int): Coût du noeud.
    """

    def __init__(self, name: str, cost: int) -> None:
        self.name = name
        self.cost = cost

    # redéfinition de la méthode __eq__ pour pouvoir comparer deux noeuds
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, Node):
            return self.name == __value.name
        return False

    # redéfinition de la méthode __lt__ pour pouvoir comparer deux noeuds en se basant sur leur coût
    def __lt__(self, __value: object) -> bool:
        if isinstance(__value, Node):
            return self.cost < __value.cost
        return False

    # redéfinition de la méthode __gt__ pour pouvoir comparer deux noeuds en se basant sur leur coût
    def __gt__(self, __value: object) -> bool:
        if isinstance(__value, Node):
            return self.cost > __value.cost
        return False

    # redéfinition de la méthode __repr__ pour pouvoir afficher un noeud
    def __repr__(self) -> str:
        return f"Node(name={self.name}, cost={self.cost})"

    # redéfinition de la méthode __str__ pour pouvoir afficher un noeud
    def __str__(self) -> str:
        return self.__repr__()

    def __hash__(self) -> int:
        return hash(self.name)

In [119]:
class SearchState:
    r"""
    Cette classe définit l'état de la recherche dans l'algorithme de recherche gloutonne.

    Attributes
    ----------
        node (Node): Noeud courant.
        parent (Node): Noeud parent.
    """

    def __init__(self, node: Node, parent: Node = None) -> None:
        self.node = node
        self.parent = parent

    def __repr__(self) -> str:
        return f"SearchState(node={self.node}, parent={self.parent})"

    def __str__(self) -> str:
        return self.__repr__()

In [120]:
Graph = dict[str, list[Node]]  # type alias pour représenter un graphe

_graph: Graph = {
    "A": [Node("B", 2), Node("C", 3), Node("E", 2)],
    "B": [Node("E", 4), Node("D", 1)],
    "C": [Node("F", 4), Node("G", 2)],
    "D": [Node("H", 0)],
    "E": [Node("D", 1), Node("H", 0)],
    "F": [],
    "G": [],
    "H": [],
}

In [121]:
from typing import Callable

# type alias pour la fonction heuristique
Heuristic = Callable[[Node, Node], int]

In [122]:
import heapq


class GreedyBestFirstSearch:
    def __init__(
        self,
        graph: Graph,
        start: Node,
        goal: Node,
        heuristic: Heuristic,
    ) -> None:
        self.graph = graph
        self.start = start
        self.goal = goal
        self.heuristic = heuristic

        # open définit les noeuds à explorer
        # cette liste est triée par coût croissant
        # raison pour laquelle on utilise le module heapq
        self._open: list[tuple[int, Node]] = []

        # visited définit les noeuds déjà explorés
        self._visited: set[Node] = set()

    def search(self):
        # on ajoute le noeud de départ à open
        self._open.append((self.heuristic(self.start, self.goal), self.start))
        states: list[SearchState] = []

        # tant qu'il y a des noeuds à explorer
        while self._open:
            # on récupère le noeud de open avec le coût le plus faible
            _, current_node = heapq.heappop(self._open)

            if current_node == self.goal:
                return self._reconstruct_path(states)

            if current_node in self._visited:
                continue

            self._visited.add(current_node)

            for neighbor in self.graph[current_node.name]:
                if neighbor not in self._open:
                    # on ajoute le nouveau noeud à explorer dans la liste open
                    heapq.heappush(
                        self._open,
                        (self.heuristic(neighbor, self.goal), neighbor),
                    )

                    states.append(SearchState(neighbor, current_node))

        return None

    def _reconstruct_path(self, states: list[SearchState]):
        # on va reconstruire le chemin par l'arrière
        current = self.goal
        path: list[Node] = []

        for state in reversed(states):
            if state.parent is not None:  # donc pas le noeud de départ
                if state.node == current:
                    path.append(current)
                    current = state.parent

        path.append(self.start)
        path.reverse()

        return path

In [123]:
def min_heuristic(node: Node, goal: Node) -> int:
    return min(node.cost, goal.cost)

In [124]:
solver = GreedyBestFirstSearch(
    graph=_graph,
    start=Node("A", 10),
    goal=Node("H", 0),
    heuristic=min_heuristic,
)

solution = solver.search()
print(solution)

[Node(name=A, cost=10), Node(name=B, cost=2), Node(name=D, cost=1), Node(name=H, cost=0)]


In [125]:
created_path = ",".join([node.name for node in solution])
print(f"Le chemin créé est: {created_path}")

Le chemin créé est: A,B,D,H
